movies filtering


In [90]:
import pandas as pd
import re
from difflib import SequenceMatcher

# Load datasets
movies = pd.read_csv('../res/movies.csv')
netflix_titles = pd.read_csv('../res/netflix_titles.csv')

print(f"Original movies dataset: {len(movies)} movies")
print(f"Netflix titles dataset: {len(netflix_titles)} titles")
print(f"\nFirst few movies:")
print(movies.head())
print(f"\nFirst few Netflix titles:")
print(netflix_titles.head())

# Function to clean and normalize text (enhanced)
def clean_text(text):
    """Clean text by removing extra whitespace, converting to lowercase, and normalizing"""
    if pd.isna(text):
        return ""
    # Convert to string and lowercase
    text = str(text).lower().strip()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters that might cause issues
    text = text.replace(':', '').replace('  ', ' ')
    # Remove common articles at the beginning
    text = re.sub(r'^(the|a|an)\s+', '', text)
    # Remove punctuation that might differ between datasets
    text = text.replace('.', '').replace(',', '').replace('!', '').replace('?', '')
    text = text.replace("'", '').replace('"', '').replace('-', ' ')
    # Clean up multiple spaces again
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Function to calculate similarity between two strings
def similarity_score(a, b):
    """Calculate similarity ratio between two strings"""
    return SequenceMatcher(None, a, b).ratio()

Original movies dataset: 87585 movies
Netflix titles dataset: 6131 titles

First few movies:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

First few Netflix titles:
    type                             title  release_year
0  Movie              Dick Johnson Is Dead          2020
1  Movie  My Little Pony: A New Generation          2021
2  Movie                           Sankofa          1993
3  Movie                      The Starling          2021


In [91]:
# Filter only movies from Netflix dataset (exclude TV shows if any)
netflix_movies = netflix_titles[netflix_titles['type'] == 'Movie'].copy()

# Clean Netflix data
netflix_movies['title_clean'] = netflix_movies['title'].apply(clean_text)
netflix_movies['release_year'] = netflix_movies['release_year'].astype(int)

# Create multiple formats for matching
# Format 1: "title (year)" - matches movies.csv format
netflix_movies['title_with_year'] = netflix_movies['title_clean'] + ' (' + netflix_movies['release_year'].astype(str) + ')'

# Format 2: Just the cleaned title without year for alternative matching
netflix_movies['title_only'] = netflix_movies['title_clean']

print(f"Netflix movies only: {len(netflix_movies)} movies")
print(f"\nSample formatted titles:")
print(netflix_movies[['title', 'release_year', 'title_with_year', 'title_only']].head(10))

Netflix movies only: 6131 movies

Sample formatted titles:
                                               title  release_year  \
0                               Dick Johnson Is Dead          2020   
1                   My Little Pony: A New Generation          2021   
2                                            Sankofa          1993   
3                                       The Starling          2021   
4                                       Je Suis Karl          2021   
5                   Confessions of an Invisible Girl          2021   
6  Europe's Most Dangerous Man: Otto Skorzeny in ...          2020   
7                                          Intrusion          2021   
8                                    Avvai Shanmughi          1996   
9       Go! Go! Cory Carson: Chrissy Takes the Wheel          2021   

                                     title_with_year  \
0                        dick johnson is dead (2020)   
1             my little pony a new generation (2021)   
2 

In [92]:
# Clean movies dataset
movies['title_clean'] = movies['title'].apply(clean_text)

# Extract title and year from movies.csv format "title (year)"
def extract_title_and_year(title_string):
    """Extract title and year from format 'Title (Year)'"""
    match = re.search(r'^(.+?)\s*\((\d{4})\)$', title_string)
    if match:
        title = clean_text(match.group(1))
        year = int(match.group(2))
        return title, year
    return clean_text(title_string), None

movies[['title_only', 'year']] = movies['title_clean'].apply(
    lambda x: pd.Series(extract_title_and_year(x))
)

print(f"Movies with extracted titles and years:")
print(movies[['title', 'title_clean', 'title_only', 'year']].head(10))
print(f"\nMovies with missing years: {movies['year'].isna().sum()}")

Movies with extracted titles and years:
                                title                         title_clean  \
0                    Toy Story (1995)                    toy story (1995)   
1                      Jumanji (1995)                      jumanji (1995)   
2             Grumpier Old Men (1995)             grumpier old men (1995)   
3            Waiting to Exhale (1995)            waiting to exhale (1995)   
4  Father of the Bride Part II (1995)  father of the bride part ii (1995)   
5                         Heat (1995)                         heat (1995)   
6                      Sabrina (1995)                      sabrina (1995)   
7                 Tom and Huck (1995)                 tom and huck (1995)   
8                 Sudden Death (1995)                 sudden death (1995)   
9                    GoldenEye (1995)                    goldeneye (1995)   

                    title_only    year  
0                    toy story  1995.0  
1                      jumanji

In [93]:
# Create lookup sets and dictionaries from Netflix for multiple matching strategies

# Strategy 1: Match on "title (year)" format
netflix_titles_with_year = set(netflix_movies['title_with_year'])

# Strategy 2: Match on title only (without year)
netflix_titles_only = set(netflix_movies['title_only'])

# Strategy 3: Match on title + year as separate fields
netflix_title_year_pairs = set(
    zip(netflix_movies['title_only'], netflix_movies['release_year'])
)

# Strategy 4: Create dictionary for fuzzy matching (title -> year)
netflix_title_to_years = {}
for _, row in netflix_movies.iterrows():
    title = row['title_only']
    year = row['release_year']
    if title not in netflix_title_to_years:
        netflix_title_to_years[title] = []
    netflix_title_to_years[title].append(year)

print(f"Netflix lookup sets created:")
print(f"  - Titles with year format: {len(netflix_titles_with_year)}")
print(f"  - Titles only: {len(netflix_titles_only)}")
print(f"  - Title-year pairs: {len(netflix_title_year_pairs)}")
print(f"  - Title to years mapping: {len(netflix_title_to_years)} unique titles")

print(f"\nSample Netflix titles with year:")
for title in list(netflix_titles_with_year)[:5]:
    print(f"  - {title}")

Netflix lookup sets created:
  - Titles with year format: 6127
  - Titles only: 6108
  - Title-year pairs: 6127
  - Title to years mapping: 6108 unique titles

Sample Netflix titles with year:
  - screwball (2018)
  - salem his sisters father (2014)
  - star trek (2009)
  - fiddler on the roof (1971)
  - city of god (2002)


In [94]:
# Enhanced multi-strategy filtering (6 strategies)
# Apply six matching strategies and combine results

print("Running multi-strategy matching...")

# Strategy 1: Match full "title (year)" format
print("  Strategy 1: Full title with year...")
matches_strategy1 = movies['title_clean'].isin(netflix_titles_with_year)

# Strategy 2: Match title only (ignoring year)
print("  Strategy 2: Title only...")
matches_strategy2 = movies['title_only'].isin(netflix_titles_only)

# Strategy 3: Match title AND year as separate fields
print("  Strategy 3: Title + year pairs...")
matches_strategy3 = movies.apply(
    lambda row: (row['title_only'], row['year']) in netflix_title_year_pairs if pd.notna(row['year']) else False,
    axis=1
)

# Strategy 4: Fuzzy match on title with year tolerance (±1 year)
print("  Strategy 4: Fuzzy year matching (±1 year)...")
def fuzzy_year_match(row):
    if pd.isna(row['year']):
        return False
    title = row['title_only']
    year = row['year']
    if title in netflix_title_to_years:
        netflix_years = netflix_title_to_years[title]
        # Check if movie year is within ±1 year of any Netflix year
        for ny in netflix_years:
            if abs(year - ny) <= 1:
                return True
    return False

matches_strategy4 = movies.apply(fuzzy_year_match, axis=1)

# Strategy 5: Match without common subtitle patterns (after :, - etc)
print("  Strategy 5: Main title only (before subtitles)...")
def extract_main_title(text):
    """Extract main title before common separators like : or -"""
    if pd.isna(text):
        return ""
    # Split on common separators and take first part
    for sep in [':', ' - ', ' – ']:
        if sep in text:
            return text.split(sep)[0].strip()
    return text

netflix_main_titles = set(netflix_movies['title_only'].apply(extract_main_title))
movies['title_main'] = movies['title_only'].apply(extract_main_title)
matches_strategy5 = movies['title_main'].isin(netflix_main_titles)

# Strategy 6: Roman numeral normalization (II -> 2, III -> 3, etc.)
print("  Strategy 6: Roman numeral normalization...")
def normalize_roman_numerals(text):
    """Convert roman numerals to arabic numbers"""
    if pd.isna(text):
        return ""
    roman_map = {
        ' ii': ' 2', ' iii': ' 3', ' iv': ' 4', ' v': ' 5',
        ' vi': ' 6', ' vii': ' 7', ' viii': ' 8', ' ix': ' 9',
        ' x': ' 10', ' xi': ' 11', ' xii': ' 12'
    }
    text_lower = text.lower()
    for roman, arabic in roman_map.items():
        # Match at word boundaries
        text_lower = re.sub(r'\b' + roman.strip() + r'\b', arabic.strip(), text_lower)
    return text_lower

netflix_normalized = set(netflix_movies['title_only'].apply(normalize_roman_numerals))
movies['title_normalized'] = movies['title_only'].apply(normalize_roman_numerals)
matches_strategy6 = movies['title_normalized'].isin(netflix_normalized)

# Combine all strategies (any match counts)
movies['is_on_netflix'] = (matches_strategy1 | matches_strategy2 | matches_strategy3 | 
                           matches_strategy4 | matches_strategy5 | matches_strategy6)

# Show matching statistics
print("\n" + "="*80)
print("MATCHING STATISTICS:")
print("="*80)
print(f"  Strategy 1 (full title with year): {matches_strategy1.sum():,} matches")
print(f"  Strategy 2 (title only): {matches_strategy2.sum():,} matches")
print(f"  Strategy 3 (title + year pairs): {matches_strategy3.sum():,} matches")
print(f"  Strategy 4 (fuzzy year ±1): {matches_strategy4.sum():,} matches")
print(f"  Strategy 5 (main title only): {matches_strategy5.sum():,} matches")
print(f"  Strategy 6 (roman numerals): {matches_strategy6.sum():,} matches")
print(f"  {'─'*80}")
print(f"  TOTAL UNIQUE MATCHES: {movies['is_on_netflix'].sum():,} matches")

# Filter to get only Netflix movies
movies_netflix = movies[movies['is_on_netflix']].copy()

print(f"\nFiltered movies (Netflix only): {len(movies_netflix):,} movies")
print(f"Percentage of Netflix dataset: {len(movies_netflix)/len(netflix_titles)*100:.2f}%")
print(f"Percentage of original movies: {len(movies_netflix)/len(movies)*100:.2f}%")

Running multi-strategy matching...
  Strategy 1: Full title with year...
  Strategy 2: Title only...
  Strategy 3: Title + year pairs...
  Strategy 4: Fuzzy year matching (±1 year)...
  Strategy 4: Fuzzy year matching (±1 year)...
  Strategy 5: Main title only (before subtitles)...
  Strategy 6: Roman numeral normalization...
  Strategy 5: Main title only (before subtitles)...
  Strategy 6: Roman numeral normalization...

MATCHING STATISTICS:
  Strategy 1 (full title with year): 3,414 matches
  Strategy 2 (title only): 5,220 matches
  Strategy 3 (title + year pairs): 3,414 matches
  Strategy 4 (fuzzy year ±1): 3,797 matches
  Strategy 5 (main title only): 5,221 matches
  Strategy 6 (roman numerals): 5,225 matches
  ────────────────────────────────────────────────────────────────────────────────
  TOTAL UNIQUE MATCHES: 5,226 matches

Filtered movies (Netflix only): 5,226 movies
Percentage of Netflix dataset: 85.24%
Percentage of original movies: 5.97%

MATCHING STATISTICS:
  Strategy 1 

In [95]:
# Show examples of matches from each strategy
print("\n" + "="*80)
print("SAMPLE MATCHES BY STRATEGY:")
print("="*80)

# Strategy 1 examples
s1_only = matches_strategy1 & ~matches_strategy2 & ~matches_strategy3 & ~matches_strategy4 & ~matches_strategy5 & ~matches_strategy6
if s1_only.sum() > 0:
    print("\nStrategy 1 (Full title+year) examples:")
    for _, row in movies[s1_only].head(3).iterrows():
        print(f"  - {row['title']}")

# Strategy 4 examples (fuzzy year)
s4_only = matches_strategy4 & ~matches_strategy1 & ~matches_strategy2 & ~matches_strategy3 & ~matches_strategy5 & ~matches_strategy6
if s4_only.sum() > 0:
    print("\nStrategy 4 (Fuzzy year ±1) examples:")
    for _, row in movies[s4_only].head(3).iterrows():
        print(f"  - {row['title']} (year: {row['year']})")

# Strategy 5 examples (main title)
s5_only = matches_strategy5 & ~matches_strategy1 & ~matches_strategy2 & ~matches_strategy3 & ~matches_strategy4 & ~matches_strategy6
if s5_only.sum() > 0:
    print("\nStrategy 5 (Main title only) examples:")
    for _, row in movies[s5_only].head(3).iterrows():
        print(f"  - {row['title']}")

# Strategy 6 examples (roman numerals)
s6_only = matches_strategy6 & ~matches_strategy1 & ~matches_strategy2 & ~matches_strategy3 & ~matches_strategy4 & ~matches_strategy5
if s6_only.sum() > 0:
    print("\nStrategy 6 (Roman numeral normalization) examples:")
    for _, row in movies[s6_only].head(3).iterrows():
        print(f"  - {row['title']}")


SAMPLE MATCHES BY STRATEGY:

Strategy 5 (Main title only) examples:
  - City of God – 10 Years Later (2013)

Strategy 6 (Roman numeral normalization) examples:
  - Pet Sematary II (1992)
  - Tremors II: Aftershocks (1996)
  - Berserk: The Golden Age Arc 2 - The Battle for Doldrey (2012)


In [96]:
# Keep only the original columns for the final dataset
movies_netflix_final = movies_netflix[['movieId', 'title', 'genres']].copy()
movies_netflix_final = movies_netflix_final.drop_duplicates(subset=['title'])

# Save the filtered dataset
output_path = '../res/movies_netflix.csv'
movies_netflix_final.to_csv(output_path, index=False)

print(f"Filtered dataset saved to: {output_path}")
print(f"\nFinal Dataset Statistics:")
print(f"  - Total movies: {len(movies_netflix_final)}")
print(f"  - Unique genres: {movies_netflix_final['genres'].str.split('|').explode().nunique()}")
print(f"\nFirst 10 filtered movies:")
print(movies_netflix_final.head(10))

Filtered dataset saved to: ../res/movies_netflix.csv

Final Dataset Statistics:
  - Total movies: 5175
  - Unique genres: 20

First 10 filtered movies:
     movieId                                 title  \
6          7                        Sabrina (1995)   
9         10                      GoldenEye (1995)   
12        13                          Balto (1995)   
26        27                   Now and Then (1995)   
37        38                   It Takes Two (1995)   
43        44                  Mortal Kombat (1995)   
45        46  How to Make an American Quilt (1995)   
60        61                 Eye for an Eye (1996)   
71        72          Kicking and Screaming (1995)   
109      111                    Taxi Driver (1976)   

                           genres  
6                  Comedy|Romance  
9       Action|Adventure|Thriller  
12   Adventure|Animation|Children  
26                 Children|Drama  
37                Children|Comedy  
43       Action|Adventure|Fantasy  
4

In [97]:
# Verify some matches with detailed comparison
print("\n" + "="*80)
print("SAMPLE MATCHED MOVIES (Random 10):")
print("="*80)
sample_movies = movies_netflix_final.sample(min(10, len(movies_netflix_final)))
for _, row in sample_movies.iterrows():
    print(f"MovieID: {row['movieId']:5d} | {row['title']:50s} | {row['genres']}")

# Show genre distribution
print("\n" + "="*80)
print("GENRE DISTRIBUTION:")
print("="*80)
genre_counts = movies_netflix_final['genres'].str.split('|').explode().value_counts()
print(genre_counts.head(15))


SAMPLE MATCHED MOVIES (Random 10):
MovieID: 26462 | Bad Boys (1983)                                    | Crime|Drama|Thriller
MovieID: 99695 | Pusher (2012)                                      | Crime|Drama
MovieID: 239110 | Sweeney Todd: The Demon Barber of Fleet Street (2014) | Drama|Horror|Thriller
MovieID: 93443 | Goon (2011)                                        | Comedy|Drama
MovieID: 181115 | Hangman (2017)                                     | Crime|Mystery|Thriller
MovieID: 157036 | The Penalty (1941)                                 | Crime|Drama
MovieID: 123691 | The Miracle (1959)                                 | Drama
MovieID: 200550 | Paranormal Investigation (2018)                    | Horror
MovieID: 208793 | Watchman (2019)                                    | Drama|Thriller
MovieID: 188513 | End Game (2018)                                    | Documentary

GENRE DISTRIBUTION:
genres
Drama                 2079
Comedy                1461
Thriller               974
Ac

ratings filtering

In [98]:
# Load the filtered Netflix movies and ratings dataset
movies_netflix_filtered = pd.read_csv('../res/movies_netflix.csv')
ratings = pd.read_csv('../res/ratings.csv')

print(f"Filtered Netflix movies: {len(movies_netflix_filtered)} movies")
print(f"Original ratings dataset: {len(ratings):,} ratings")
print(f"\nRatings columns: {list(ratings.columns)}")
print(f"\nFirst few ratings:")
print(ratings.head())

Filtered Netflix movies: 5175 movies
Original ratings dataset: 32,000,204 ratings

Ratings columns: ['userId', 'movieId', 'rating', 'timestamp']

First few ratings:
   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976
3       1       30     5.0  944249077
4       1       32     5.0  943228858


In [99]:
# Create a set of valid movieIds from the filtered Netflix movies
valid_movie_ids = set(movies_netflix_filtered['movieId'])

print(f"Valid Netflix movie IDs: {len(valid_movie_ids)} unique IDs")
print(f"\nSample valid movie IDs: {list(valid_movie_ids)[:10]}")

Valid Netflix movie IDs: 5175 unique IDs

Sample valid movie IDs: [7, 163849, 10, 13, 131090, 196627, 196629, 229397, 27, 229413]


In [100]:
# Filter ratings to only include ratings for Netflix movies
ratings_netflix = ratings[ratings['movieId'].isin(valid_movie_ids)].copy()

print(f"Filtered ratings (Netflix movies only): {len(ratings_netflix):,} ratings")
print(f"Original ratings: {len(ratings):,}")
print(f"Ratings removed: {len(ratings) - len(ratings_netflix):,}")
print(f"Percentage retained: {len(ratings_netflix)/len(ratings)*100:.2f}%")

print(f"\nUnique users in filtered ratings: {ratings_netflix['userId'].nunique():,}")
print(f"Unique movies in filtered ratings: {ratings_netflix['movieId'].nunique():,}")

Filtered ratings (Netflix movies only): 4,624,438 ratings
Original ratings: 32,000,204
Ratings removed: 27,375,766
Percentage retained: 14.45%

Unique users in filtered ratings: 199,595
Unique movies in filtered ratings: 5,127


In [101]:
# Save the filtered ratings dataset
output_path = '../res/ratings_netflix.csv'
ratings_netflix.to_csv(output_path, index=False)

print(f"Filtered ratings saved to: {output_path}")
print(f"\nFiltered ratings statistics:")
print(f"  - Total ratings: {len(ratings_netflix):,}")
print(f"  - Unique users: {ratings_netflix['userId'].nunique():,}")
print(f"  - Unique movies: {ratings_netflix['movieId'].nunique():,}")
print(f"  - Average rating: {ratings_netflix['rating'].mean():.2f}")
print(f"  - Rating distribution:")
print(ratings_netflix['rating'].value_counts().sort_index())

Filtered ratings saved to: ../res/ratings_netflix.csv

Filtered ratings statistics:
  - Total ratings: 4,624,438
  - Unique users: 199,595
  - Unique movies: 5,127
  - Average rating: 3.56
  - Rating distribution:
rating
0.5      78906
1.0     125761
1.5      81208
2.0     272548
2.5     254204
3.0     815386
3.5     660667
4.0    1200254
4.5     477341
5.0     658163
Name: count, dtype: int64


In [82]:
# Verify the filtering - check some sample movies
print("="*80)
print("SAMPLE RATINGS FOR NETFLIX MOVIES:")
print("="*80)

# Get a few sample movies and show their ratings
sample_movie_ids = ratings_netflix['movieId'].value_counts().head(5).index
for movie_id in sample_movie_ids:
    movie_title = movies_netflix_filtered[movies_netflix_filtered['movieId'] == movie_id]['title'].values[0]
    movie_ratings = ratings_netflix[ratings_netflix['movieId'] == movie_id]
    print(f"\nMovie: {movie_title} (ID: {movie_id})")
    print(f"  Number of ratings: {len(movie_ratings)}")
    print(f"  Average rating: {movie_ratings['rating'].mean():.2f}")
    print(f"  Rating range: {movie_ratings['rating'].min():.1f} - {movie_ratings['rating'].max():.1f}")

SAMPLE RATINGS FOR NETFLIX MOVIES:

Movie: Pulp Fiction (1994) (ID: 296)
  Number of ratings: 98409
  Average rating: 4.20
  Rating range: 0.5 - 5.0

Movie: Schindler's List (1993) (ID: 527)
  Number of ratings: 73849
  Average rating: 4.24
  Rating range: 0.5 - 5.0

Movie: American Beauty (1999) (ID: 2858)
  Number of ratings: 65158
  Average rating: 4.09
  Rating range: 0.5 - 5.0

Movie: Inception (2010) (ID: 79132)
  Number of ratings: 57931
  Average rating: 4.16
  Rating range: 0.5 - 5.0

Movie: Dances with Wolves (1990) (ID: 590)
  Number of ratings: 46771
  Average rating: 3.73
  Rating range: 0.5 - 5.0
